In [2]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
import sys

from nltk.corpus import PlaintextCorpusReader
import os
import sys
wordlists = PlaintextCorpusReader("Nepali_Corpus", '.*txt')
data = wordlists.fileids()[:10]
text = []
invalid = 0
for i in data:
    with open(os.path.join("Nepali_Corpus",i),encoding='utf8') as file:
        try:
            text.append(file.read())
        except UnicodeDecodeError :
            print(i)
            invalid+=1
print(invalid)
raw_text = "".join(text)
raw_text = raw_text.replace("\n","")

0


In [25]:

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 200
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = np_utils.to_categorical(X)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model


[' ', '(', ')', ',', '.', '?', '\xa0', '¥', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ए', 'ऐ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', '्', '।', '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', '–', '‘', '’']
Total Characters:  13424
Total Vocab:  78
Total Patterns:  13224


In [4]:
X.shape

(13224, 200, 78)

In [5]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2,callbacks=callbacks_list)

Epoch 1/50
13224/13224 [==============================] - 191s 14ms/step - loss: 3.4924

Epoch 00001: loss improved from inf to 3.49244, saving model to weights-improvement-01-3.4924-bigger.hdf5
Epoch 2/50
13224/13224 [==============================] - 187s 14ms/step - loss: 3.1520

Epoch 00002: loss improved from 3.49244 to 3.15199, saving model to weights-improvement-02-3.1520-bigger.hdf5
Epoch 3/50
13224/13224 [==============================] - 187s 14ms/step - loss: 2.8391

Epoch 00003: loss improved from 3.15199 to 2.83912, saving model to weights-improvement-03-2.8391-bigger.hdf5
Epoch 4/50
13224/13224 [==============================] - 187s 14ms/step - loss: 2.6856

Epoch 00004: loss improved from 2.83912 to 2.68561, saving model to weights-improvement-04-2.6856-bigger.hdf5
Epoch 5/50
13224/13224 [==============================] - 188s 14ms/step - loss: 2.5354

Epoch 00005: loss improved from 2.68561 to 2.53541, saving model to weights-improvement-05-2.5354-bigger.hdf5
Epoch 6/5

13224/13224 [==============================] - 186s 14ms/step - loss: 0.1284

Epoch 00042: loss improved from 0.12989 to 0.12842, saving model to weights-improvement-42-0.1284-bigger.hdf5
Epoch 43/50
13224/13224 [==============================] - 186s 14ms/step - loss: 0.1150

Epoch 00043: loss improved from 0.12842 to 0.11500, saving model to weights-improvement-43-0.1150-bigger.hdf5
Epoch 44/50
13224/13224 [==============================] - 187s 14ms/step - loss: 0.1244

Epoch 00044: loss did not improve from 0.11500
Epoch 45/50
13224/13224 [==============================] - 186s 14ms/step - loss: 0.1233

Epoch 00045: loss did not improve from 0.11500
Epoch 46/50
13224/13224 [==============================] - 186s 14ms/step - loss: 0.1197

Epoch 00046: loss did not improve from 0.11500
Epoch 47/50
13224/13224 [==============================] - 188s 14ms/step - loss: 0.1151

Epoch 00047: loss did not improve from 0.11500
Epoch 48/50
13224/13224 [==============================] - 186s 

In [8]:
model.save("2-layer-200-weights-one-hot.h5")

In [10]:
model = keras.models.load_model("2-layer-200-weights-one-hot.h5")

In [68]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = X[start]
print(pattern)
print ("Seed:")
print (''.join([int_to_char[numpy.argmax(value)] for value in pattern]), "\n")
print("".join([int_to_char[numpy.argmax(value)] for value in pattern]),end="")
# generate characters
f = open("sent_dump","w")
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), -1))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    print(result,sep='',end='')
#     f.write("".join([int_to_char[numpy.argmax(j)] 
#             for i in x[0]
#                for j in i
#            ])+"-->"+result+"\n")
    next = np_utils.to_categorical(index,num_classes=n_vocab).reshape(1,-1)    
    pattern = numpy.vstack((pattern,next))
    pattern = pattern[1:,:]
print ("\nDone.")
f.close()

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Seed:
े त्यसबेला आफूले बाल्यावस्थामा कविताबाटै झगडा गर्ने गरेको स्मरण गरिन् । ‘बाबाका कृतिहरू यसरी एकमुष्ठ प्रकाशित भएपछि अब उहाँलाई पढ्न सजिलो भएको छ,’ डा. पाण्डेले भनिन् । कृतिका सम्पादक प्राध्यापक सनतकुम 

े त्यसबेला आफूले बाल्यावस्थामा कविताबाटै झगडा गर्ने गरेको स्मरण गरिन् । ‘बाबाका कृतिहरू यसरी एकमुष्ठ प्रकाशित भएपछि अब उहाँलाई पढ्न सजिलो भएको छ,’ डा. पाण्डेले भनिन् । कृतिका सम्पादक प्राध्यापक सनतकुमारले आफू लेखनाथको साहित्यको पुजारी रहेकाले उनको कृतिमा काम गर्न पाउनु मनले रुचाएको अनुष्ठान गर्नु रहेको जनाए । उनले नै लेखनाथको रचनाको खोजबिन गरेर कृतिलाई पूर्णता दिएका हुन् । पुस्तकलाई नेपालयले प्रकाशन गरेको हो।  कामा अभिनेता राईलाई आइतबार सद्भावना दूत बनाइएको संग्रहालय निर्माण समितिका संयोजक खुक्सङ खम्बुले बताए । काठमाडौं – गीतकार तथा गजलकार कल्पना श्रेष्ठ कल्पपुञ्जले ‘सञ्जीवनी’ उपन्यास लिएर पाठकमाझ आएकी छन । शुक्रबार राजधानीमा ए